In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
import re
import contractions
import nltk
import wordsegment
# Initialize the wordsegment library
wordsegment.load()

In [2]:
paragraph = []

In [3]:
def recur_element(element):
    # print(element.tag, element.text)
    # if element.tag == 'H1' or element.tag == 'H2' or element.tag == 'H3' or element.tag == 'TOCI':
    #     paragraph.append(element.text)
    if element.tag == 'P' and len(element.text.split()) > 5:
        paragraph.append(element.text)
    elif element.tag == 'L':
        text = ''
        for list_element in element:
            text += recur_element(list_element)
        paragraph.append(text)
    elif element.tag == 'LI':
        return element.text
    for sub_child in element:
        recur_element(sub_child)

In [4]:
# Load the XML file
%cd ~/muku_esg/ESG-BERT/
tree = ET.parse('./xml_american/4_apple_1_72.xml')

# Get the root element of the XML document
root = tree.getroot()

recur_element(root)
print(len(paragraph))
# for p in paragraph:
#     print(p)

/home/kai/muku_esg/ESG-BERT
813


In [5]:
def preprocess(x):
    x = x.replace('\n', ' ')
    x = re.sub(r'\’', '\'', x)
    x = contractions.fix(x)
    x = re.sub(r'[^\x00-\x7F]', '', x)
    x = re.sub(r'[\x0C]', r"\n\n", x)
    x = re.sub(r'[!\"#$%&\'()*\+,-.\/:;<=>?@\[\\\]^_`{|}~]', '', x)
    x = x.replace('\t', ' ')
    x = x.strip()
    x = ' '.join(wordsegment.segment(x))
    return x

In [25]:
# # Define the concatenated string
# concatenated_string = "strongencryption recognizingthat softwareand anddevelopment withmaximum userexperience keepingpersonal designedthe "

# # Perform word segmentation
# segmented_string = " ".join(wordsegment.segment(concatenated_string))

# # Print the result
# print(segmented_string)


strong encryption recognizing that software and and development with maximum user experience keeping personal designed the


In [6]:
for i in range(100):
    # print('before', paragraph[i])
    paragraph[i] = preprocess(paragraph[i])
    # print('after', paragraph[i])

before 3Letter from Tim Cook4Report highlights6Our approach8Our commitmentto transparency9Advocating for change10Our commitmentto human rights
after 3letterfromtimcook4 report highlights 6ourapproach8 our commitment to transparency 9advocatingforchange10 our commitment to human rights
before 13Our approach13Climate change18Resources20Smarter chemistry
after 13ourapproach13 climate change 18resources20 smarter chemistry
before 23Our approach23Inclusion and diversity26Growth anddevelopment27Benefits28Compensation29Engagement30Workplace practicesand policies33Health and safetyat Apple
after 23ourapproach23 inclusion and diversity 26 growth and development 27benefits28 compensation 29engagement30 workplace practices and policies 33 health and safety at apple
before 37Our approach40Labor and human rightsin the supply chain43Health, safety,and wellness44Responsible materialssourcing45Education andprofessional development46Environment
after 37ourapproach40 labor and human rights in the supply

KeyboardInterrupt: 

In [ ]:
paragraph = [x for x in paragraph if re.fullmatch(r'[0-9]*', x) == None]
print(len(paragraph))

In [ ]:
for x in paragraph:
    print(x)

In [ ]:
%cd predict_american
with open("./output.txt", "w") as file:
    file.truncate()
with open("./paragraph.txt", "w") as file:
    file.truncate()
for i in range(len(paragraph)):
    # Open a file in write mode
    with open("./paragraph.txt", "w") as f:
        # Write the strings to the file
        f.writelines(paragraph[i])
    # os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt');
    os.system(f'curl -X POST http://127.0.0.1:8090/predictions/bert -T paragraph.txt >> output.txt')
    os.system(f'echo >> output.txt')

In [ ]:
csv_folder = 'predict_american'
type_folder = 'xml'
file_name = '4_apple_1_72'

In [ ]:
with open('./output.txt', 'r') as f:
    labels = f.readlines()
    labels = [x.strip('\n') for x in labels]
ans = pd.DataFrame({'paragraph': paragraph, 'predict_label': labels})
ans.to_csv(f'../{csv_folder}/{type_folder}/{file_name}.csv', sep=',', index=False, encoding='utf-8')

# Appendix

In [ ]:
# def recur_element(element):
#     # print(element.tag, element.text)
#     if element.tag == 'H1' or element.tag == 'H2' or element.tag == 'H3' or element.tag == 'TOCI':
#         paragraph.append(element.text)
#     for sub_child in element:
#         recur_element(sub_child)